<div class="alert alert-block alert-success">
    <h1>
        Example notebook - Integrate RDF file
    </h1>
    <p>
        Link to dataset : <a href="TR/owl-guide/wine.rdf">Link to Wine RDF file download</a>
    </p>
</div>

# Import modules and functions

In [1]:
from rdflib import Graph as RDFGraph
import re
import time

In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

# Check data files are available

In [3]:
example_name = "wine_ontology"
path_data = f"{os.getcwd()}/data/{example_name}"
if not os.path.exists(path_data):
    print(f"{path_data} does not exist")
    os.makedirs(path_data)
    print(f"{path_data} folder created ✔")

list_files = sorted(os.listdir(path_data))
rdf_filename = "wine_ontology.rdf"
if not rdf_filename in list_files:
    print(f"Wine ontology RDF file is not available in {path_data}. \n")
    url_rdf = "https://www.w3.org/TR/owl-guide/wine.rdf"
    print(
        f"Downloading from: {url_rdf}"
        "\n...\n"
    )
    os.system(f"wget {url_rdf} -O {path_data}/{rdf_filename}")

# Load dataset

In [4]:
path_rdf = f"{path_data}/{example_name}.rdf"
path_rdf

'/home/dev/turingdb-examples/examples/notebooks/public_version/data/wine_ontology/wine_ontology.rdf'

# Graph Creation in TuringDB

## Build Cypher CREATE Commands

In [5]:
def sanitize_identifier(s):
    """Sanitize identifiers (labels, relationship types, property names)"""
    if not s:
        return "ID_empty"
    
    # Replace invalid characters with underscores
    s = s.replace('#', '_').replace('-', '_').replace('/', '_').replace('.', '_')
    
    # Prefix with ID_ if starts with digit or underscore
    if s[0].isdigit() or s[0] == '_':
        s = 'ID_' + s
    
    # Remove any other non-alphanumeric characters except underscores
    s = ''.join(c if c.isalnum() or c == '_' else '_' for c in s)
    
    return s


def sanitize_value(s):
    """Sanitize property values (escape quotes)"""
    return s.replace("'", "\\'").replace('"', '\\"')

In [6]:
def owlready_to_cypher(rdf_file, include_blank_nodes=False):
    from owlready2 import get_ontology, ThingClass, Thing, ObjectProperty, DataProperty, sync_reasoner_pellet
    
    onto = get_ontology(f"file://{rdf_file}").load()
    
    # Run reasoner
    try:
        with onto:
            sync_reasoner_pellet(infer_property_values=True, infer_data_property_values=True)
        print("Reasoner completed successfully")
    except Exception as e:
        print(f"Reasoner failed: {e}, continuing without reasoning")
    
    node_parts = []
    edge_parts = []
    node_counter = 0
    node_to_var = {}
    
    def get_var(uri):
        nonlocal node_counter
        if uri not in node_to_var:
            node_to_var[uri] = f"n{node_counter}"
            node_counter += 1
        return node_to_var[uri]
    
    def get_most_generic_parent(cls):
        """Get the most generic non-Thing parent"""
        ancestors = [a for a in cls.ancestors() 
                    if isinstance(a, ThingClass) 
                    and a.name != 'Thing'
                    and a != cls]
        return ancestors[-1] if ancestors else cls
    
    # First pass: collect all nodes that will be created
    valid_nodes = set()
    
    # Add all classes
    for cls in onto.classes():
        if cls.name != 'Thing':
            valid_nodes.add(cls.iri)
    
    # Add all properties
    for prop in onto.properties():
        valid_nodes.add(prop.iri)
    
    # Add all individuals
    for ind in onto.individuals():
        valid_nodes.add(ind.iri)
    
    # Create nodes for all classes
    for cls in onto.classes():
        if cls.name == 'Thing':
            continue
        
        var = get_var(cls.iri)
        root = get_most_generic_parent(cls)
        label = sanitize_identifier(root.name)
        name = cls.name
        
        # Build properties
        props = [
            f"uri: '{sanitize_value(cls.iri)}'",
            f"name: '{sanitize_value(name)}'",
            f"type: 'Class'"
        ]
        
        # Add direct parent names (only those in valid_nodes)
        direct_parents = [p.name for p in cls.is_a 
                         if isinstance(p, ThingClass) 
                         and p.name != 'Thing'
                         and p.iri in valid_nodes]
        if direct_parents:
            props.append(f"parents: '{sanitize_value(", ".join(direct_parents))}'")
        
        # Add comment if exists
        if hasattr(cls, 'comment') and cls.comment:
            comment = cls.comment[0] if isinstance(cls.comment, list) else cls.comment
            props.append(f"comment: '{sanitize_value(str(comment))}'")
        
        # Add label if exists
        if hasattr(cls, 'label') and cls.label:
            lbl = cls.label[0] if isinstance(cls.label, list) else cls.label
            props.append(f"rdfs_label: '{sanitize_value(str(lbl))}'")
        
        node_parts.append(f"({var}:{label} {{{', '.join(props)}}})")
    
    # Track added edges to avoid duplicates
    added_edges = set()
    
    # Create subClassOf relationships (only for valid nodes)
    for cls in onto.classes():
        if cls.name == 'Thing':
            continue
        
        s_var = get_var(cls.iri)
        
        for parent in cls.is_a:
            if isinstance(parent, ThingClass) and parent.name != 'Thing' and parent.iri in valid_nodes:
                o_var = get_var(parent.iri)
                edge = f"({s_var})-[:subClassOf]->({o_var})"
                if edge not in added_edges:
                    edge_parts.append(edge)
                    added_edges.add(edge)
    
    # Create nodes for properties
    for prop in onto.properties():
        prop_var = get_var(prop.iri)
        prop_type = "ObjectProperty" if isinstance(prop, ObjectProperty) else "DataProperty"
        
        props_list = [
            f"uri: '{sanitize_value(prop.iri)}'",
            f"name: '{sanitize_value(prop.name)}'",
            f"type: 'Property'",
            f"property_type: '{prop_type}'"
        ]
        
        node_parts.append(f"({prop_var}:Property {{{', '.join(props_list)}}})")
        
        # Add domain relationships (only for valid nodes)
        if hasattr(prop, 'domain') and prop.domain:
            domains = prop.domain if isinstance(prop.domain, list) else [prop.domain]
            for domain in domains:
                if isinstance(domain, ThingClass) and domain.name != 'Thing' and domain.iri in valid_nodes:
                    domain_var = get_var(domain.iri)
                    edge = f"({domain_var})-[:hasDomain]->({prop_var})"
                    if edge not in added_edges:
                        edge_parts.append(edge)
                        added_edges.add(edge)
        
        # Add range relationships (only for valid nodes)
        if hasattr(prop, 'range') and prop.range:
            ranges = prop.range if isinstance(prop.range, list) else [prop.range]
            for rng in ranges:
                if isinstance(rng, ThingClass) and rng.name != 'Thing' and rng.iri in valid_nodes:
                    range_var = get_var(rng.iri)
                    edge = f"({prop_var})-[:hasRange]->({range_var})"
                    if edge not in added_edges:
                        edge_parts.append(edge)
                        added_edges.add(edge)
    
    # Handle individuals (only for valid nodes)
    for ind in onto.individuals():
        s_var = get_var(ind.iri)
        
        for prop in ind.get_properties():
            values = getattr(ind, prop.name, [])
            if not isinstance(values, list):
                values = [values]
            
            for value in values:
                if isinstance(value, Thing) and value.iri in valid_nodes:
                    o_var = get_var(value.iri)
                    pred = sanitize_identifier(prop.name)
                    edge = f"({s_var})-[:{pred}]->({o_var})"
                    if edge not in added_edges:
                        edge_parts.append(edge)
                        added_edges.add(edge)
    
    # Combine nodes first, then edges
    return "CREATE " + ",\n".join(node_parts + edge_parts)

In [7]:
%%time

graph_CREATE_command_owlready = owlready_to_cypher(path_rdf, include_blank_nodes=True)
print(graph_CREATE_command_owlready)

# TODO: Remove when v2 ready
# Sanitise query for v1
def sanitise_for_v1(query):
    query = query.replace("->", "-")
    
    return query

graph_CREATE_command_owlready = sanitise_for_v1(graph_CREATE_command_owlready)

# Save Cypher query to file
with open(f"{path_data}/{example_name}_owlready.cypher", "w") as f:
    f.write(graph_CREATE_command_owlready)

graph_CREATE_command = graph_CREATE_command_owlready

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /home/dev/turingdb-examples/.venv/lib/python3.13/site-packages/owlready2/pellet/antlr-3.2.jar:/home/dev/turingdb-examples/.venv/lib/python3.13/site-packages/owlready2/pellet/jena-core-2.10.0.jar:/home/dev/turingdb-examples/.venv/lib/python3.13/site-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/home/dev/turingdb-examples/.venv/lib/python3.13/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/dev/turingdb-examples/.venv/lib/python3.13/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/dev/turingdb-examples/.venv/lib/python3.13/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/dev/turingdb-examples/.venv/lib/python3.13/site-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/home/dev/turingdb-examples/.venv/lib/python3.13/site-packages/owlready2/pellet/jgrapht-jdk1.5.jar:/home/dev/turingdb-examples/.venv/lib/python3.13/site-packages/owlready2/pellet/log4j-core-2.19.0.jar:/home/dev/turingdb-examples/.

Reasoner completed successfully
CREATE (n0:PotableLiquid {uri: 'http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine##Wine', name: 'Wine', type: 'Class', rdfs_label: 'wine'}),
(n1:Winery {uri: 'http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine##Winery', name: 'Winery', type: 'Class'}),
(n2:Vintage {uri: 'http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine##Vintage', name: 'Vintage', type: 'Class'}),
(n3:Grape {uri: 'http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine##WineGrape', name: 'WineGrape', type: 'Class'}),
(n4:Wine {uri: 'http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine##WhiteWine', name: 'WhiteWine', type: 'Class', parents: 'Wine'}),
(n5:Wine {uri: 'http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine##WhiteTableWine', name: 'WhiteTableWine', type: 'Class', parents: 'DryWine, TableWine, WhiteNonSweetWine'}),
(n6:Wine {uri: 'http://www.w3.org/TR/2003/PR-owl-guide-20031209/wine##TableWine', name: 'TableWine', type: 'Class', parents: 'Wine'}),
(n7:Wine {uri: 'http://www

* Owlready2 * Pellet took 1.3653736114501953 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


# Create graph using `turingdb` python package

<div class="alert alert-block alert-info">
    <h2>
        See <a href="https://docs.turingdb.ai/quickstart">TuringDB Get started documentation</a> for the important steps to follow :
    </h2>
    <h3>
        <ul>
            <li>Create your TuringDB account</li>
            <li>Create your instance in the <a href="https://console.turingdb.ai/auth">TuringDB Cloud UI</a></li>
            <li>Copy your Instance ID from the Database Instances management page</li>
            <li>Get API Key from the Settings in UI</li>
        </ul>
        Remember to have your instance active while working in this notebook !
    </h3>
</div>

In [8]:
from turingdb import TuringDB

# Create TuringDB client
client = TuringDB(
    host="http://localhost:6666"  # Remove this parameter and set the two parameters below
    # instance_id=os.getenv("INSTANCE_ID"),
    # auth_token=os.getenv("AUTH_TOKEN"),
)

In [9]:
%%time

client.s3_connect(
    bucket_name="turing-internal",
    region="eu-west-2",
    access_key=os.getenv("AWS_ACCESS_KEY"),
    secret_key=os.getenv("AWS_SECRET_KEY"),
)

CPU times: user 145 ms, sys: 51.9 ms, total: 197 ms
Wall time: 284 ms


In [10]:
# Get list of loaded graphs
list_graphs = client.list_loaded_graphs()
list_graphs

['default']

In [11]:
# Set graph name
graph_name_prefix = example_name
graph_name_nb_suffix = str(
    max(
        [
            int(re.sub(graph_name_prefix, "", g))
            for g in list_graphs
            if g.startswith(graph_name_prefix)
            and re.sub(graph_name_prefix, "", g).isdigit()
        ]
        + [0]
    )
    + 1
)
graph_name = graph_name_prefix + graph_name_nb_suffix
graph_name = re.sub("-", "_", graph_name)
print(f"graph_name: {graph_name}")

graph_name: wine_ontology1


In [12]:
from turingdb.exceptions import TuringDBException

In [13]:
%%time

# Set graph
try:
    client.create_graph(graph_name)
except TuringDBException as e:
    print(e)

# Set working graph
client.set_graph(graph_name)

CPU times: user 1.57 ms, sys: 34 μs, total: 1.6 ms
Wall time: 10.8 ms


In [14]:
%%time

# Create a new change on the graph
client.checkout()
change = client.new_change()
print(f"Current change {change}")

# Checkout into the change
client.checkout(change=change)

Current change 0
CPU times: user 1.18 ms, sys: 274 μs, total: 1.45 ms
Wall time: 1.27 ms


In [15]:
%%time

# Run CREATE command
print("\nExecuting query on TuringDB...")
start_time = time.time()
result = client.query(graph_CREATE_command)
execution_time = time.time() - start_time
print(f"✓ Graph created successfully in {execution_time:.2f} seconds")

# Commit the change
client.query("COMMIT")
client.query("CHANGE SUBMIT")

# Checkout into main
client.checkout()


Executing query on TuringDB...
✓ Graph created successfully in 0.00 seconds
CPU times: user 3.08 ms, sys: 5 μs, total: 3.08 ms
Wall time: 62 ms


# Query TuringDB

## Use metaqueries to have insight on graph overall structure

<h3>
    To learn more about 📮 Metaqueries, please check TuringDB documentation on this <a href="https://turingdb.mintlify.app/query/cypher_subset#%F0%9F%93%AE-metaqueries">link</a>
</h3>

In [16]:
%%time

# CALL PROPERTIES() - returns a column of all the different node and edge properties and their types in the database
command = """
CALL PROPERTIES()
"""
df_PROPERTIES = client.query(command)
if df_PROPERTIES.empty:
    print("No result found")
else:
    df_PROPERTIES.columns = ["Property_ID", "Property_name", "Property_type"]
    display(df_PROPERTIES)

,Property_ID,Property_name,Property_type
0,0,uri,String
1,1,name,String
2,2,type,String
3,3,rdfs_label,String
4,4,parents,String
5,5,comment,String
6,6,property_type,String


CPU times: user 4.41 ms, sys: 1.96 ms, total: 6.37 ms
Wall time: 5.68 ms


In [17]:
# Get node properties
nodes_properties = df_PROPERTIES["Property_name"].values.tolist()
print(f"Node properties: {nodes_properties}")

Node properties: ['uri', 'name', 'type', 'rdfs_label', 'parents', 'comment', 'property_type']


In [18]:
%%time

# CALL LABELS () - returns a column of all the different node labels
command = """
CALL LABELS()
"""
df_LABELS = client.query(command)
if df_LABELS.empty:
    print("No result found")
else:
    df_LABELS.columns = ["Node_type_ID", "Node_label"]
    display(df_LABELS)

,Node_type_ID,Node_label
0,0,PotableLiquid
1,1,Winery
2,2,Vintage
3,3,Grape
4,4,Wine
5,5,Region
6,6,VintageYear
7,7,WineDescriptor
8,8,RedWine
9,9,Property


CPU times: user 3.22 ms, sys: 951 μs, total: 4.17 ms
Wall time: 3.55 ms


In [19]:
%%time

# CALL EDGETYPES() - returns a column of all the different edge types (edge equivalent of node labels)
command = """
CALL EDGETYPES()
"""
df_EDGETYPES = client.query(command)
if df_EDGETYPES.empty:
    print("No result found")
else:
    df_EDGETYPES.columns = ["Edge_type_ID", "Edge_label"]
    display(df_EDGETYPES)

,Edge_type_ID,Edge_label
0,0,subClassOf
1,1,hasDomain
2,2,hasRange


CPU times: user 4.35 ms, sys: 1.9 ms, total: 6.24 ms
Wall time: 5.61 ms


In [20]:
%%time

# CALL LABELSETS() - returns a two columns describing combinations of node labels
command = """
CALL LABELSETS()
"""
df_LABELSETS = client.query(command)
if df_LABELSETS.empty:
    print("No result found")
else:
    df_LABELSETS.columns = ["Node_type_ID", "Node_label"]
    display(df_LABELSETS)

,Node_type_ID,Node_label
0,0,PotableLiquid
1,1,Winery
2,2,Vintage
3,3,Grape
4,4,Wine
5,5,Region
6,6,VintageYear
7,7,WineDescriptor
8,8,RedWine
9,9,Property


CPU times: user 6.22 ms, sys: 951 μs, total: 7.17 ms
Wall time: 6.31 ms


In [21]:
%%time

# Find number of nodes and number of edges in the graph
n_nodes = len(client.query("MATCH (n) RETURN n"))
n_edges = len(client.query("MATCH (n)--(m) RETURN n, m"))
print(f"Graph: {n_nodes:,} nodes and {n_edges:,} edges")

Graph: 87 nodes and 135 edges
CPU times: user 2.46 ms, sys: 1.03 ms, total: 3.49 ms
Wall time: 2.82 ms


## Simple queries

In [22]:
from turingdb_examples.utils import get_return_statements

In [23]:
%%time

# Match all edges and return them
command = """
MATCH (n)-[e]-(m)
RETURN n.name, n.type, e, m.name, m.type
"""
df_all_edges = client.query(command)
if df_all_edges.empty:
    print("No result found")
else:
    df_all_edges.columns = get_return_statements(command)
    display(df_all_edges)

,n.name,n.type,e,m.name,m.type
0,Wine,Class,0,hasWineDescriptor,Property
1,Wine,Class,1,hasColor,Property
2,Wine,Class,2,madeFromGrape,Property
3,CaliforniaWine,Class,3,Wine,Class
4,Loire,Class,4,Wine,Class
...,...,...,...,...,...
130,hasColor,Property,130,WineColor,Class
131,hasBody,Property,131,WineBody,Class
132,hasFlavor,Property,132,WineFlavor,Class
133,hasSugar,Property,133,WineSugar,Class


CPU times: user 8.77 ms, sys: 50 μs, total: 8.82 ms
Wall time: 8.16 ms


In [24]:
# Get all nodes by label type
for label in df_LABELS["Node_label"]:
    print(100 * "#")
    print(f"label: {label}")
    display(client.query(f"MATCH (n:{label}) RETURN n.name"))

print(100 * "#")

####################################################################################################
label: PotableLiquid


,0
0,Wine
1,CaliforniaWine
2,Loire


####################################################################################################
label: Winery


,0
0,Winery


####################################################################################################
label: Vintage


...

####################################################################################################
label: RedWine


,0
0,Beaujolais
1,DryRedWine
2,CotesDOr
3,Chianti
4,CabernetFranc
5,CabernetSauvignon
6,Zinfandel
7,StEmilion
8,RedTableWine
9,RedBurgundy


####################################################################################################
label: Property


,0
0,producesWine
1,hasWineDescriptor
2,madeIntoWine
3,adjacentRegion
4,hasVintageYear
5,locatedIn
6,hasColor
7,hasBody
8,hasFlavor
9,hasSugar


####################################################################################################


## Complex queries

In [25]:
%%time

# 1. Get all wine types (classes that are subclasses of Wine/PotableLiquid)
#command = """
#MATCH (wine:PotableLiquid)
#WHERE wine.type = 'Class'
#RETURN wine.name, wine.parents
#LIMIT 20
#"""
command = """
MATCH (wine:PotableLiquid {type: 'Class'})
RETURN wine.name, wine.parents
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,wine.name,wine.parents
0,Wine,None
1,CaliforniaWine,Wine
2,Loire,Wine


CPU times: user 3.54 ms, sys: 38 μs, total: 3.58 ms
Wall time: 2.96 ms


In [26]:
%%time

# 2. Find all properties and their domains/ranges
#command = """
#MATCH (domain)-[:hasDomain]->(prop:Property)-[:hasRange]->(range)
#RETURN domain.name, prop.name, prop.property_type, range.name
#"""
command = """
MATCH (domain)-[:hasDomain]-(prop:Property)-[:hasRange]-(range)
RETURN domain.name, prop.name, prop.property_type, range.name
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,domain.name,prop.name,prop.property_type,range.name
0,Wine,hasWineDescriptor,DataProperty,WineDescriptor
1,Wine,hasColor,DataProperty,WineColor
2,Wine,madeFromGrape,DataProperty,WineGrape
3,Vintage,hasVintageYear,DataProperty,VintageYear
4,Region,adjacentRegion,DataProperty,Region


CPU times: user 4.13 ms, sys: 13 μs, total: 4.15 ms
Wall time: 3.6 ms


In [27]:
%%time

# 3. Get the complete class hierarchy for a specific wine (e.g., Chardonnay)
#command = """
#MATCH path = (leaf)-[:subClassOf*]->(root)
#WHERE leaf.name = 'Chardonnay'
#RETURN leaf.name, root.name, LENGTH(path) AS depth
#ORDER BY depth DESC
#"""
command = """
MATCH (leaf {name: 'Chardonnay'})-[:subClassOf]-(root)
RETURN leaf.name, root.name
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,leaf.name,root.name
0,Chardonnay,WhiteWine


CPU times: user 1.66 ms, sys: 1.92 ms, total: 3.58 ms
Wall time: 2.91 ms


In [28]:
%%time

# 4. Find all red wines (wines that are subclass of RedWine)
#command = """
#MATCH (wine)-[:subClassOf*]->(parent)
#WHERE parent.name = 'RedWine'
#RETURN wine.name, wine.parents
#"""
command = """
MATCH (wine)-[:subClassOf]-(parent {name: 'RedWine'})
RETURN wine.name, wine.parents
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,wine.name,wine.parents
0,DryRedWine,"RedWine, DryWine, TableWine"
1,RedTableWine,"RedWine, DryWine, TableWine"
2,RedBordeaux,"Bordeaux, RedWine"
3,Port,"RedWine, FullBodiedWine, SweetWine"
4,PinotNoir,RedWine
5,Meritage,RedWine


CPU times: user 3.63 ms, sys: 36 μs, total: 3.67 ms
Wall time: 3.04 ms


In [29]:
%%time

# 5. Find all direct children of Wine class
#command = """
#MATCH (child)-[:subClassOf]->(parent)
#WHERE parent.name = 'Wine'
#RETURN child.name, child.type
#"""
command = """
MATCH (child)-[:subClassOf]-(parent {name: 'Wine'})
RETURN child.name, child.type
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,child.name,child.type
0,CaliforniaWine,Class
1,Loire,Class
2,TexasWine,Class
3,RedWine,Class
4,RoseWine,Class
5,LateHarvest,Class
6,EarlyHarvest,Class
7,ItalianWine,Class
8,GermanWine,Class
9,Gamay,Class


CPU times: user 3.83 ms, sys: 54 μs, total: 3.88 ms
Wall time: 3.25 ms


In [30]:
%%time

# 6. Get all properties that Wine class can have
#command = """
#MATCH (wine)-[:hasDomain]->(prop:Property)
#WHERE wine.name = 'Wine'
#RETURN prop.name, prop.property_type
#"""
command = """
MATCH (wine {name: 'Wine'})-[:hasDomain]-(prop:Property)
RETURN prop.name, prop.property_type
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,prop.name,prop.property_type
0,hasWineDescriptor,DataProperty
1,hasColor,DataProperty
2,madeFromGrape,DataProperty


CPU times: user 3.49 ms, sys: 31 μs, total: 3.52 ms
Wall time: 2.9 ms


In [31]:
%%time

# 7. Find wine classes that have the most parent classes (complex definitions)
#command = """
#MATCH (wine)-[:subClassOf]->(parent)
#WHERE wine.type = 'Class'
#WITH wine.name AS wine_name, COUNT(parent) AS parent_count
#RETURN wine_name, parent_count
#ORDER BY parent_count DESC
#LIMIT 10
#"""
command = """
MATCH (wine {type: 'Class'})-[:subClassOf]-(parent)
RETURN wine.name, parent.name
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,wine.name,parent.name
0,CaliforniaWine,Wine
1,Loire,Wine
2,Chardonnay,WhiteWine
3,PinotBlanc,WhiteWine
4,TexasWine,Wine
...,...,...
115,Merlot,RedTableWine
116,Merlot,DryRedWine
117,Meritage,RedWine
118,Margaux,Merlot


CPU times: user 1.43 ms, sys: 2.9 ms, total: 4.34 ms
Wall time: 3.64 ms


In [32]:
%%time

# 8. Find all French wine types
#command = """
#MATCH (wine)-[:subClassOf*]->(parent)
#WHERE parent.name = 'FrenchWine'
#RETURN wine.name
#"""
command = """
MATCH (wine)-[:subClassOf]-(parent {name: 'FrenchWine'})
RETURN wine.name
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

No result found
CPU times: user 475 μs, sys: 962 μs, total: 1.44 ms
Wall time: 954 μs


In [33]:
%%time

# 9. Find properties that point to WineDescriptor classes
#command = """
#MATCH (prop:Property)-[:hasRange]->(desc)
#WHERE desc.name CONTAINS 'Wine' AND desc.type = 'Class'
#RETURN prop.name, desc.name
#"""
command = """
MATCH (prop:Property)-[:hasRange]-(desc {name ~= 'Wine'})
RETURN prop.name, desc.name
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,prop.name,desc.name
0,hasWineDescriptor,WineDescriptor
1,hasColor,WineColor
2,hasBody,WineBody
3,hasFlavor,WineFlavor
4,hasSugar,WineSugar
5,madeFromGrape,WineGrape


CPU times: user 2.67 ms, sys: 979 μs, total: 3.65 ms
Wall time: 2.99 ms


In [34]:
%%time

# 10. Get the full subclass tree depth for all wine types
#command = """
#MATCH path = (wine)-[:subClassOf*]->(root:PotableLiquid)
#WHERE wine.type = 'Class' AND root.name = 'Wine'
#RETURN wine.name, LENGTH(path) AS hierarchy_depth
#ORDER BY hierarchy_depth DESC
#LIMIT 15
#"""
command = """
MATCH (wine {type: 'Class'})-[:subClassOf]-(root:PotableLiquid {name: 'Wine'})
RETURN wine.name, root.name
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = get_return_statements(command)
    display(df)

,wine.name,root.name
0,CaliforniaWine,Wine
1,Loire,Wine
2,TexasWine,Wine
3,RedWine,Wine
4,RoseWine,Wine
5,LateHarvest,Wine
6,EarlyHarvest,Wine
7,ItalianWine,Wine
8,GermanWine,Wine
9,Gamay,Wine


CPU times: user 4 ms, sys: 65 μs, total: 4.06 ms
Wall time: 3.3 ms


In [35]:
print("Notebook finished !")

Notebook finished !
